## 1. Importar librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltd
import seaborn as sns

from gensim.models import Word2Vec

C:\Users\JAVI\Miniconda3\envs\dhds\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2. Cargar datos

In [3]:
movies=pd.read_csv('../Data/movies.csv')
ratings=pd.read_csv('../Data/ratings.csv')
links=pd.read_csv('../Data/links.csv')
tags=pd.read_csv('../Data/tags.csv')
imdb=pd.read_csv('../Data/tab_imdb_info.csv',index_col=0)

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [7]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


In [8]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


## 3. Feature Extraction

Corregir id's de imdb:

In [9]:
imdb['imdbId']=imdb['req_imdbid'].apply(lambda x: x[2:]).astype('int64')

In [10]:
pd.merge(links,movies,on='movieId').head()

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,Father of the Bride Part II (1995),Comedy


In [11]:
imdb.isna().sum()

actors        32
country       12
director      94
req_imdbid     0
year           0
imdbId         0
dtype: int64

In [12]:
imdb.fillna('-',inplace=True)

In [13]:
imdb['actors'].apply(lambda x: len(x.split(','))).unique()

array([4, 2, 3, 1], dtype=int64)

In [14]:
# imdb['director1'] = imdb['director'].apply(lambda x: x.split(',')[0])
# imdb['actor1'] = imdb['actors'].apply(lambda x: x.split(',')[0])
# #imdb['actor2'] = imdb['actors'].apply(lambda x: x.split(',')[1])
# imdb['country1'] = imdb['country'].apply(lambda x: x.split(',')[0])

Creo un dataframe con todos los datos necesarios para crear features por películas:

In [15]:
mov_with_feat = pd.merge(links,imdb,on='imdbId')
mov_with_feat = pd.merge(mov_with_feat,movies,on='movieId')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862.0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


In [16]:
mov_with_feat.isna().sum()

movieId       0
imdbId        0
tmdbId        8
actors        0
country       0
director      0
req_imdbid    0
year          0
title         0
genres        0
dtype: int64

In [17]:
mov_with_feat.fillna('-',inplace=True)

In [18]:
mov_with_feat['actors'] = mov_with_feat['actors'].str.replace(' ','')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace('|',',')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace(' ','')
mov_with_feat['director'] = mov_with_feat['director'].str.replace(' ','')
mov_with_feat['country'] = mov_with_feat['country'].str.replace(' ','')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy"
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy"
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance"
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance"
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


Armado de tokens:

In [19]:
#df1 = mov_with_feat[['actor1','director1','country1']]
df1 = mov_with_feat[['actors','director','country','genres']]
df1 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)
df1 = df1.str.replace(' ','')
df1 = pd.DataFrame({'clean': df1}) 
print(df1.iloc[0].values)
sent = [row.split(',') for row in df1['clean']]
mov_sent = [a for a in zip(imdb['req_imdbid'],sent)]
mov_sent[0:2]

['TomHanks,TimAllen,DonRickles,JimVarney,JohnLasseter,USA,Adventure,Animation,Children,Comedy,Fantasy']


[('tt0114709',
  ['TomHanks',
   'TimAllen',
   'DonRickles',
   'JimVarney',
   'JohnLasseter',
   'USA',
   'Adventure',
   'Animation',
   'Children',
   'Comedy',
   'Fantasy']),
 ('tt0113497',
  ['RobinWilliams',
   'JonathanHyde',
   'KirstenDunst',
   'BradleyPierce',
   'JoeJohnston',
   'USA',
   'Adventure',
   'Children',
   'Fantasy'])]

Entrenar modelo de word2vec de gensim:

In [20]:
model = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [21]:
#Representation for director John Lasseter
model.wv['JohnLasseter']

array([-0.06513455, -0.05357993, -0.00784492,  0.04903268, -0.06228677,
        0.01729047,  0.0332482 ,  0.09084351,  0.01733334, -0.00576951,
       -0.03562678,  0.04957668, -0.04490826,  0.00513787,  0.05696142,
       -0.05547569,  0.04508258,  0.03508759,  0.00611551,  0.02993434,
       -0.00450447,  0.02810488,  0.03586949,  0.15793872,  0.05700679,
       -0.0807839 ,  0.02346669, -0.00485344,  0.03263455,  0.00918836,
       -0.00370202, -0.11537883,  0.079957  , -0.08854255,  0.0391321 ,
       -0.10401874, -0.00099702, -0.03302311, -0.01729025,  0.00545133,
        0.01726113, -0.06727697, -0.09599549, -0.00858708, -0.04017727,
        0.09036271,  0.0054476 , -0.03321884,  0.04880426,  0.01844932],
      dtype=float32)

In [22]:
#Get similarity
model.wv.similarity('TomHanks', 'USA')

0.5269329130397457

In [23]:
model.wv['Adventure']

array([-0.15005015, -0.6116287 ,  0.27079964,  0.17159277,  0.08979527,
        0.13976486,  0.4936723 ,  0.5582854 , -0.5274856 , -0.2662793 ,
        0.28853273, -0.03253898, -0.63796437, -0.30863243,  0.8837206 ,
       -0.9834892 ,  0.10290851,  0.6208423 ,  0.5248298 , -0.09030478,
        0.01872242, -0.06732021, -0.20051268,  0.35983455,  0.34918037,
       -0.94554234,  0.04577538,  0.04378659,  0.14848171,  0.49827707,
        0.17155252, -0.26868334,  0.17600599, -0.49367732,  0.4610561 ,
       -1.1534053 ,  0.68705136, -0.41333264,  0.36980993,  1.2927449 ,
        0.1987763 , -0.2891264 , -0.62387824, -0.11485973,  0.20993064,
        0.4365118 ,  0.25655055,  0.12777013,  0.80288285,  0.34236103],
      dtype=float32)

In [24]:
model.wv['(nogenreslisted)']

array([-0.09162746, -0.07461325, -0.00069353,  0.06562842, -0.08128706,
        0.00244307,  0.06321871,  0.14696223,  0.003079  ,  0.00334302,
       -0.04582416,  0.09201556, -0.13142928,  0.04177121,  0.09909967,
       -0.07026197,  0.06139214,  0.04451076,  0.01334166,  0.04127597,
       -0.01395735,  0.04129964,  0.07111347,  0.2907442 ,  0.0991927 ,
       -0.14109369,  0.0363652 ,  0.01529003,  0.05573293,  0.01140377,
        0.01243316, -0.20780341,  0.12498571, -0.1447764 ,  0.08556683,
       -0.17399745,  0.01649323, -0.06696954, -0.01969743,  0.02157802,
        0.0521393 , -0.12681334, -0.18081841, -0.03112936, -0.05201434,
        0.17350696,  0.00518498, -0.06709794,  0.12190174,  0.05328292],
      dtype=float32)

In [25]:
imdb.head()

,actors,country,director,req_imdbid,year,imdbId
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,114709
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,113497
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,113228
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,114885
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,113041


In [26]:
imdb.dtypes

actors        object
country       object
director      object
req_imdbid    object
year          object
imdbId         int64
dtype: object

In [27]:
def rep_col(text):
    #print(text)
    rep = []
    for item in text.split(','):
        rep.append(model.wv[item])
    return np.concatenate(rep,axis=0)

In [28]:
def items_rep(item_lst):
    a=0
    for item in item_lst:
        a+=model.wv[item]
    a/=len(item_lst)
    return a



In [29]:
mov_with_feat['actors_rep'] = mov_with_feat['actors'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['director_rep'] = mov_with_feat['director'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['country_rep'] = mov_with_feat['country'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['genres_rep'] = mov_with_feat['genres'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres,actors_rep,director_rep,country_rep,genres_rep
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy","[-0.098627284, -0.06452264, -0.014179099, 0.07...","[-0.06513455, -0.053579926, -0.00784492, 0.049...","[0.1606117, -0.7556332, 0.5231987, 0.18509997,...","[-0.1554276, -0.55280864, 0.2540639, 0.1599902..."
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy","[-0.08367961, -0.060699154, -0.009460866, 0.05...","[-0.04231859, -0.049106013, 0.0006052975, 0.04...","[0.1606117, -0.7556332, 0.5231987, 0.18509997,...","[-0.13878484, -0.590914, 0.2287424, 0.17225717..."
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance","[-0.10054105, -0.061897784, -0.01615449, 0.073...","[-0.03906891, -0.025711477, 0.00096076186, 0.0...","[0.1606117, -0.7556332, 0.5231987, 0.18509997,...","[-0.073302165, -0.40967262, 0.42492777, 0.1652..."
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance","[-0.070090026, -0.043671522, -0.008246271, 0.0...","[-0.09651796, -0.078760765, 0.013412462, 0.073...","[0.1606117, -0.7556332, 0.5231987, 0.18509997,...","[-0.032006145, -0.4394165, 0.48523116, 0.18079..."
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy,"[-0.119018815, -0.08060948, -0.0168114, 0.0861...","[-0.030895295, -0.025455315, 0.0036606686, 0.0...","[0.1606117, -0.7556332, 0.5231987, 0.18509997,...","[-0.1523373, -0.4479573, 0.4607106, 0.16200452..."


In [30]:
# mov_with_feat['actors_rep'][0]

In [31]:
# mov_with_feat['director_rep'][0]

In [32]:
# mov_with_feat['country_rep'][0]

In [33]:
# np.concatenate([mov_with_feat['actors_rep'][0],mov_with_feat['director_rep'][0],mov_with_feat['country_rep'][0]],axis=0)

In [34]:
mov_with_feat['all_feat']=mov_with_feat.apply(lambda x: np.concatenate([x['actors_rep'],x['director_rep'],x['country_rep'],x['genres_rep']],axis=0),axis=1)

In [35]:
mov_with_feat[['movieId','all_feat']].head()

,movieId,all_feat
0,1,"[-0.098627284, -0.06452264, -0.014179099, 0.07..."
1,2,"[-0.08367961, -0.060699154, -0.009460866, 0.05..."
2,3,"[-0.10054105, -0.061897784, -0.01615449, 0.073..."
3,4,"[-0.070090026, -0.043671522, -0.008246271, 0.0..."
4,5,"[-0.119018815, -0.08060948, -0.0168114, 0.0861..."


In [36]:
a = mov_with_feat['all_feat'][0].reshape(1,-1)
for mov_feat in mov_with_feat['all_feat'][1:]:
    b = mov_feat.reshape(1,-1)
    a = np.concatenate([a,b])

a.shape

(9742, 200)

In [37]:
#temp_actor = rep_col(imdb['actors'][0])
#print(np.concatenate(temp_actor,axis=0))
#temp_director = rep_col(imdb['director'][0])
#temp_country = rep_col(imdb['country'][0])
#a = np.concatenate(temp_actor,axis=0)
##np.concatenate([temp_actor,temp_director,temp_country],axis=1)
#np.concatenate(temp_director,axis=0)

In [38]:
#def get_movie_features(fila):
#    temp_actor = rep_col(fila['actors'])
#    a = np.concatenate(temp_actor,axis=0)
#    temp_director = rep_col(fila['director'])
#    b = np.concatenate(temp_director,axis=0)
#    temp_country = rep_col(fila['country'])
#    c = np.concatenate(temp_country,axis=0)
#    feat_lst = [a,b,c]
#    return np.concatenate(feat_lst)
#get_movie_features(imdb.iloc[0])

#imdb['actors'].apply(rep_col)

In [39]:
# imdb['feat_str'] = imdb['actor1'] + ', ' + imdb['director1'] + ', ' + imdb['country1']
# imdb['feat_str'] = imdb['feat_str'].str.replace(' ','')
# imdb.head()

In [40]:
# imdb['feat_str'][0]

In [41]:
# imdb['movie_feat'] = imdb['feat_str'].apply(rep_col)
# imdb.head(5)

In [42]:
# mov_with_feat['genres'] = mov_with_feat['genres'].str.replace('|',',')
# mov_with_feat['feat_str'] = mov_with_feat['actor1'] + ', ' + mov_with_feat['director1'] + ', ' + mov_with_feat['country1'] #+ ', ' + mov_with_feat['genres']  
# mov_with_feat['feat_str'] = mov_with_feat['feat_str'].str.replace(' ','')
# mov_with_feat.head()

In [43]:
# mov_with_feat['features'] = mov_with_feat['feat_str'].apply(rep_col)
# mov_with_feat[['movieId','features']].head()

In [44]:
# a = mov_with_feat['features'][0].reshape(1,-1)
# for mov_feat in mov_with_feat['features'][1:]:
#     b = mov_feat.reshape(1,-1)
#     a = np.concatenate([a,b])

# a.shape

In [45]:
# np.median(a)

In [46]:
# np.mean(a)

In [47]:
# lista = [sublista.split('|') for sublista in movies['genres']]
# genres = sorted(set([item for s in lista for item in s]))
# dummies = pd.DataFrame(np.zeros((len(movies), len(genres)), dtype=int), columns=genres)
# for i, gen in enumerate(movies['genres']):
#     dummies.loc[i, gen.split('|')] = 0.2
# dummies.values

In [48]:
# dummies.shape

## 4. Obtención de peliculas similares

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

#movie_features = np.concatenate([a,dummies],axis=1)
movie_features = a
cosine_sim = cosine_similarity(movie_features,dense_output=True)
cosine_sim[0:5]

array([[1.        , 0.9988209 , 0.9871794 , ..., 0.9694684 , 0.94616294,
        0.9799886 ],
       [0.9988209 , 1.        , 0.9819624 , ..., 0.9627937 , 0.94305193,
        0.97482246],
       [0.9871794 , 0.9819624 , 1.        , ..., 0.9887344 , 0.92918354,
        0.99186003],
       [0.98138034, 0.97567844, 0.99384785, ..., 0.9950658 , 0.92939776,
        0.9883979 ],
       [0.9849663 , 0.9777984 , 0.99569714, ..., 0.9902432 , 0.94460243,
        0.9951565 ]], dtype=float32)

In [50]:
# movies.loc[movies['movieId'] == 1,'title'].values[0]

In [51]:
results = {}

for idx, row in movies.iterrows():
    similar_indices = cosine_sim[idx].argsort()[:-100:-1]
    similar_items = [(cosine_sim[idx][i], movies['movieId'][i]) for i in similar_indices]
    results[row['movieId']] = similar_items[1:]

## 5. Recomendar películas similares a una película en particular

In [52]:
def item(mid):
    return movies.loc[movies['movieId'] == mid]['title'].tolist()[0].split(' - ')[0]

def recommend_movie(item_id, num):
    print("Recomendando " + str(num) + " productos similares a " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        
#Ingresando el ID de la película y el número de recomendaciones devuelve los títulos mejor posisionados

In [53]:
recommend_movie(1,10)

Recomendando 10 productos similares a Toy Story (1995)...
-------
Recomendado: Madagascar (2005) (score:0.99971133)
Recomendado: Ice Age 2: The Meltdown (2006) (score:0.99963653)
Recomendado: The Boss Baby (2017) (score:0.9995539)
Recomendado: Antz (1998) (score:0.9995277)
Recomendado: Minions (2015) (score:0.9993278)
Recomendado: Home (2015) (score:0.99930304)
Recomendado: Monsters, Inc. (2001) (score:0.9993009)
Recomendado: Cool World (1992) (score:0.99925977)
Recomendado: Over the Hedge (2006) (score:0.99925643)
Recomendado: Emperor's New Groove, The (2000) (score:0.9992494)


In [54]:
recommend_movie(648,10)

Recomendando 10 productos similares a Mission: Impossible (1996)...
-------
Recomendado: Death Proof (2007) (score:0.9995393)
Recomendado: Island, The (2005) (score:0.99947894)
Recomendado: Darkman (1990) (score:0.99938744)
Recomendado: Matrix, The (1999) (score:0.99937904)
Recomendado: Blade Runner (1982) (score:0.9992826)
Recomendado: Captain America: The First Avenger (2011) (score:0.9992516)
Recomendado: One, The (2001) (score:0.9992028)
Recomendado: Chronicles of Riddick, The (2004) (score:0.9992007)
Recomendado: Scanner Darkly, A (2006) (score:0.9991232)
Recomendado: Hackers (1995) (score:0.9990895)


In [55]:
recommend_movie(2492,10)

Recomendando 10 productos similares a 20 Dates (1998)...
-------
Recomendado: The Escort (2015) (score:0.99990624)
Recomendado: Outsourced (2006) (score:0.9999056)
Recomendado: Lola Versus (2012) (score:0.99989456)
Recomendado: How to Make Love to a Woman (2010) (score:0.9998936)
Recomendado: It Happened on Fifth Avenue (1947) (score:0.9998914)
Recomendado: Mischief (1985) (score:0.9998781)
Recomendado: Play the Game (2009) (score:0.9998723)
Recomendado: Nina Takes a Lover (1994) (score:0.99985975)
Recomendado: Sex Ed (2014) (score:0.9998594)
Recomendado: Lost & Found (1999) (score:0.99985415)


In [56]:
# imdb['actors'].apply(lambda x: len(x.split(','))).unique()

### Lista de películas vistas por usuario

In [57]:
ratings_sorted = ratings.sort_values(by='timestamp')
movies_by_user = pd.DataFrame(ratings_sorted.groupby('userId')['movieId'].apply(list))
movies_by_user.head()

,movieId
userId,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744..."
2,"[318, 79132, 131724, 115713, 99114, 112552, 35..."
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3..."
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5..."
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ..."


Para cada usuario me quedo con la última película que vio:

In [58]:
movies_by_user['movieId'][1][-1]

2492

In [59]:
movies_by_user['last_watched'] = movies_by_user['movieId'].apply(lambda x: x[-1])
movies_by_user.head()

,movieId,last_watched
userId,,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744...",2492
2,"[318, 79132, 131724, 115713, 99114, 112552, 35...",80489
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3...",2424
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5...",4246
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ...",474


Y le recomiendo las 10 películas más parecidas a esa, sin recomendarle alguna que ya vio:

In [60]:
def user_recommendation(uid=1,num=1):
    last_watched_id = movies_by_user.loc[uid]['last_watched']
    movies_watched = movies_by_user.loc[uid]['movieId']
    recommendation = results[last_watched_id]#[:num]
    i=0
    j=0
    recs=[]
    while i<num:
        if recommendation[j][1] not in movies_watched:
            recs.append(recommendation[j])
            j+=1
            i+=1
        else:
            j+=1
    
    #print("Recomendando " + str(num) + " productos similares a " + item(last_watched_id) + "...")
    print("Porque ya viste " + item(last_watched_id) + "...")
    print("-------")
    for rec in recs:
        #print("Recomendado: " + str(rec[1]) + ' ' + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
    #return recommendation

In [61]:
uid=610
num=10
user_recommendation(uid,num)

Porque ya viste Hellraiser (1987)...
-------
Recomendado: Hellbound: Hellraiser II (1988) (score:0.99960595)
Recomendado: The Plague of the Zombies (1966) (score:0.9994568)
Recomendado: Horror of Dracula (Dracula) (1958) (score:0.998719)
Recomendado: Awakening, The (1980) (score:0.9984838)
Recomendado: Curse of Frankenstein, The (1957) (score:0.9983713)
Recomendado: Mummy, The (1959) (score:0.9983321)
Recomendado: Black Mirror: White Christmas (2014) (score:0.9973811)
Recomendado: Legend of Hell House, The (1973) (score:0.99707943)
Recomendado: The Hound of the Baskervilles (1988) (score:0.9970715)
Recomendado: The Deadly Bees (1967) (score:0.9966915)


In [62]:
uid=1
num=10
user_recommendation(uid,num)

Porque ya viste 20 Dates (1998)...
-------
Recomendado: The Escort (2015) (score:0.99990624)
Recomendado: Outsourced (2006) (score:0.9999056)
Recomendado: Lola Versus (2012) (score:0.99989456)
Recomendado: How to Make Love to a Woman (2010) (score:0.9998936)
Recomendado: It Happened on Fifth Avenue (1947) (score:0.9998914)
Recomendado: Mischief (1985) (score:0.9998781)
Recomendado: Play the Game (2009) (score:0.9998723)
Recomendado: Nina Takes a Lover (1994) (score:0.99985975)
Recomendado: Sex Ed (2014) (score:0.9998594)
Recomendado: Lost & Found (1999) (score:0.99985415)


In [63]:
uid=154
num=10
user_recommendation(uid,num)

Porque ya viste Finding Dory (2016)...
-------
Recomendado: Planes (2013) (score:0.9998964)
Recomendado: ParaNorman (2012) (score:0.9997401)
Recomendado: Dragons: Gift of the Night Fury (2011) (score:0.99951994)
Recomendado: Cars 3 (2017) (score:0.9994921)
Recomendado: Planes: Fire & Rescue (2014) (score:0.9993984)
Recomendado: Puss in Boots: The Three Diablos (2012) (score:0.99934965)
Recomendado: Wreck-It Ralph (2012) (score:0.9992212)
Recomendado: Ice Age 4: Continental Drift (2012) (score:0.99920255)
Recomendado: Black Knight (2001) (score:0.99919)
Recomendado: Alvin and the Chipmunks: Chipwrecked (2011) (score:0.99895793)


In [64]:
# sorted(movies_by_user.loc[uid]['movieId'])

User Random Recommendation (tomo una pelicula random que vio, y le recomiendo otras):

In [66]:
import random

def user_rand_recommendation(uid=1,num=1):
    movies_watched = movies_by_user.loc[uid]['movieId']
    rand_watched_id = random.choice(movies_watched)
    recommendation = results[rand_watched_id]#[:num]
    i=0
    j=0
    recs=[]
    while i<num:
        if recommendation[j][1] not in movies_watched:
            recs.append(recommendation[j])
            j+=1
            i+=1
        else:
            j+=1
    
    #print("Recomendando " + str(num) + " productos similares a " + item(last_watched_id) + "...")
    print("Porque ya viste " + item(rand_watched_id) + "...")
    print("-------")
    for rec in recs:
        #print("Recomendado: " + str(rec[1]) + ' ' + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
    #return recommendation

In [72]:
uid=610
num=10
user_rand_recommendation(uid,num)

Porque ya viste Field in England, A (2013)...
-------
Recomendado: Paperhouse (1988) (score:0.99751496)
Recomendado: I Know Where I'm Going! (1945) (score:0.9970697)
Recomendado: Dam Busters, The (1955) (score:0.9967695)
Recomendado: Dorian Gray (2009) (score:0.9967139)
Recomendado: Red Shoes, The (1948) (score:0.9967045)
Recomendado: Henry V (1989) (score:0.9964687)
Recomendado: Pink Floyd: The Wall (1982) (score:0.99643403)
Recomendado: Abominable Snowman, The (Abominable Snowman of the Himalayas, The) (1957) (score:0.9963002)
Recomendado: Road to Guantanamo, The (2006) (score:0.9960692)
Recomendado: Edge of Love, The (2008) (score:0.99591374)


In [73]:
uid=1
num=10
user_rand_recommendation(uid,num)

Porque ya viste That Thing You Do! (1996)...
-------
Recomendado: Unstrung Heroes (1995) (score:0.9993211)
Recomendado: The Disaster Artist (2017) (score:0.99873585)
Recomendado: Bronx Tale, A (1993) (score:0.9984861)
Recomendado: Pollock (2000) (score:0.9983051)
Recomendado: Antwone Fisher (2002) (score:0.9982897)
Recomendado: Man Without a Face, The (1993) (score:0.9982252)
Recomendado: Larry Crowne (2011) (score:0.9981845)
Recomendado: Mr. Saturday Night (1992) (score:0.99787503)
Recomendado: Sling Blade (1996) (score:0.9971619)
Recomendado: Rocky Balboa (2006) (score:0.9971091)


In [74]:
uid=154
num=10
user_rand_recommendation(uid,num)

Porque ya viste Finding Dory (2016)...
-------
Recomendado: Planes (2013) (score:0.9998964)
Recomendado: ParaNorman (2012) (score:0.9997401)
Recomendado: Dragons: Gift of the Night Fury (2011) (score:0.99951994)
Recomendado: Cars 3 (2017) (score:0.9994921)
Recomendado: Planes: Fire & Rescue (2014) (score:0.9993984)
Recomendado: Puss in Boots: The Three Diablos (2012) (score:0.99934965)
Recomendado: Wreck-It Ralph (2012) (score:0.9992212)
Recomendado: Ice Age 4: Continental Drift (2012) (score:0.99920255)
Recomendado: Black Knight (2001) (score:0.99919)
Recomendado: Alvin and the Chipmunks: Chipwrecked (2011) (score:0.99895793)
